In [ ]:
# collect_vsi_metadata(base_histology_dir= "/Volumes/LaCie/data/rEO_rYL_quad/histology",
#                          subfolder = "20X",
#                          output_csv = "metadata.csv")

In [ ]:
def parse_metadata(metadatadf, filename):
    cx = metadatadf['column (x)'].loc[metadatadf['filename_electrode']==filename].values[0]
    cy = metadatadf['row (y)'].loc[metadatadf['filename_electrode']==filename].values[0]
    v_x = metadatadf['vessel_x'].loc[metadatadf['filename_electrode']==filename].values[0]
    v_y = metadatadf['vessel_y'].loc[metadatadf['filename_electrode']==filename].values[0]
    v_r = np.round(metadatadf['vessel_R'].loc[metadatadf['filename_electrode']==filename].values[0]/0.325*1.5).astype(int)
    
    r_base = metadatadf['electrode_R'].loc[metadatadf['filename_electrode']==filename].values[0]
    r_base = r_base + 10
    
    z_start = metadatadf['plane_start'].loc[metadatadf['filename_electrode']==filename].values[0]
    z_stop = metadatadf['plane_stop'].loc[metadatadf['filename_electrode']==filename].values[0]
    plane_selected = metadatadf['plane_selected'].loc[metadatadf['filename_electrode']==filename].values[0]
    
    control_location = metadatadf['control'].loc[metadatadf['filename_electrode']==filename].values[0]

    return cx, cy, v_x, v_y, v_r, r_base, z_start, z_stop, plane_selected, control_location


In [3]:
def read_metadata(path):
    filenames=[]   # Filenames
    x=[]   # Row indices for electrode centers
    y=[]   # Column indices for electrode centers
    r_base=[]
    # vessel location and size to avoid
    v_x=[]
    v_y=[]
    v_r=[]
    # start and stop planes
    plane_start=[]
    plane_stop=[]
    # array to exclude the file from analysis
    exclude=[]
    control_location=[]    
    plane_selected=[]
    
    with open(path, newline='', encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            filenames.append(row['filename'])
            x.append(int(row['row']))
            y.append(int(row['column']))

            v_x.append(int(row['vessel_x']))
            v_y.append(int(row['vessel_y']))
            v_r.append(int(row['vessel_r']))
            # start and stop planes
            plane_start.append(int(row['plane_start']))
            plane_stop.append(int(row['plane_stop']))
            plane_selected.append(int(row['plane_selected']))
            # array to exclude the file from analysis
            exclude.append(int(row['exclude']))
            control_location.append(int(row['control']))
            r_base.append(int(row['rbase']))
    # dictionary of lists 
    listdict = {'filename': filenames, 'center_x': x, 'center_y': y, 
               'v_x': v_x, 'v_y': v_y, 'v_r': v_r,
                'plane_start': plane_start, 'plane_stop': plane_stop, 'plane_selected': plane_selected,
                'exclude':exclude, 'r_base':r_base, 'control_location':control_location
               }
    df = pd.DataFrame(listdict)
       
    return df

In [ ]:
from pathlib import Path
import csv

def collect_vsi_metadata(base_histology_dir: str = "/Volumes/LaCie/data/rEO_rYL_penta/histology",
                         subfolder: str = "20X",
                         output_csv: str = "metadata.csv"):
    """
    Find .vsi files in the given histology subfolder, categorize them into 'Electrode' or 'Control',
    and save the results into a CSV file.

    Parameters
    ----------
    base_histology_dir : str
        Base directory where histology data is stored.
    subfolder : str
        Subdirectory (relative to base_histology_dir) to search for .vsi files.
    output_csv : str
        Filename of the output CSV (saved in base_histology_dir).
    """

    base_dir = Path(base_histology_dir)
    search_dir = base_dir / subfolder
    out_csv = base_dir / output_csv

    vsi_files = sorted([p for p in search_dir.iterdir() if p.is_file() and p.suffix.lower() == ".vsi"])

    filename_electrode = []
    filename_control = []
    
    for p in vsi_files:
        tokens = p.stem.split("_")
        tokens_lower = [t.lower() for t in tokens]

        if "electrode" in tokens_lower:
            filename_electrode.append(p.name)
        elif "control" in tokens_lower:
            filename_control.append(p.name)
    
    csvdict ={
        "filename_electrode": filename_electrode,
        "filename_control": filename_control,
        "column (x)": np.zeros((len(filename_control,))),
        "row (y)": np.zeros((len(filename_control,))),
        "vessel_x": np.zeros((len(filename_control,))),
        "vessel_y": np.zeros((len(filename_control,))),
        "vessel_R": np.zeros((len(filename_control,))),
        "plane_start": np.zeros((len(filename_control,))),
        "plane_stop": np.zeros((len(filename_control,))),
        "exclude": np.zeros((len(filename_control,))),
        "electrode_R": np.zeros((len(filename_control,))),
        "control": np.zeros((len(filename_control,))),
        "plane_selected": np.ones((len(filename_control,)))*11,
    }
    df = pd.DataFrame(csvdict)
    df.to_csv(out_csv, index=False)
    
    return {
        "electrode": filename_electrode,
        "control": filename_control,
        "csv_path": str(out_csv)
    }